# Text mining assignment 2 (Emma Vonk and Julius Ruijgrok)

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer
import datasets

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, num_labels=5)

c:\Users\Juliu\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Question 2
Convert the IOB data to the correct data structure for token classification in Huggingface
(words and labels like the conll2023 data in the tutorial) and align the labels with the tokens.

In [2]:
# Function to make the dataset to the correct huggingface structure explained in: https://huggingface.co/learn/nlp-course/chapter7/2?fw=pt


def read_bio_file(filepath):
    sentences = []
    current_sentence = {"tokens": [], "ner_tags": [], "pos_tags": []}

    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line == "":
                if current_sentence["tokens"]:
                    sentences.append(current_sentence)
                    current_sentence = {"tokens": [], "ner_tags": [], "pos_tags": []}
            else:
                token, pos, label = line.split()  # Each line is a token POS-label
                current_sentence["tokens"].append(token)
                # Map labels to an integer ID
                current_sentence["ner_tags"].append(label_to_id(label))
                current_sentence["pos_tags"].append(pos)

        # Add the last sentence if file doesn't end with a blank line
        if current_sentence["tokens"]:
            sentences.append(current_sentence)

    return sentences

def label_to_id(label):
    # This function should map each label to a unique integer (e.g., B-PER -> 0, I-PER -> 1, O -> 2).
    label_mapping = {"O": 0, "B-PER": 1, "I-PER": 2, "B-LOC": 3, "I-LOC": 4, 'B-ART': 5, 'I-ART': 6, 'I-MAT': 7, 'B-MAT': 8, 'I-CON': 9, 'B-CON': 10, 'I-SPE': 11, 'B-SPE': 12}
    return label_mapping.get(label, -100)  # Return -100 for unknown labels

# Read datasets
train_data = read_bio_file("train.txt")
val_data = read_bio_file("val.txt")
test_data = read_bio_file("test.txt")

# Load into HuggingFace dataset structure
dataset = datasets.DatasetDict({
    "train": datasets.Dataset.from_dict({"tokens": [d["tokens"] for d in train_data], "ner_tags": [d["ner_tags"] for d in train_data], "pos_tags": [d["pos_tags"] for d in train_data]}),
    "validation": datasets.Dataset.from_dict({"tokens": [d["tokens"] for d in val_data], "ner_tags": [d["ner_tags"] for d in val_data], "pos_tags": [d["pos_tags"] for d in val_data]}),
    "test": datasets.Dataset.from_dict({"tokens": [d["tokens"] for d in test_data], "ner_tags": [d["ner_tags"] for d in test_data], "pos_tags": [d["pos_tags"] for d in test_data
                                                                                                                                                ]})
})

# Define label mapping
label_names = ['O', 'B-PER', 'I-PER', 'B-LOC', 'I-LOC', 'B-ART', 'I-ART', 'I-MAT', 'B-MAT', 'I-CON', 'B-CON', 'I-SPE', 'B-SPE']

# Create the ClassLabel feature with only the names (otherwise the number does not overlap)
ner_feature = datasets.ClassLabel(names=label_names)

dataset = dataset.cast_column("ner_tags", datasets.Sequence(ner_feature))

Casting the dataset:   0%|          | 0/1992 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/850 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/864 [00:00<?, ? examples/s]

In [3]:
# print the structure of the data
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'pos_tags'],
        num_rows: 1992
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'pos_tags'],
        num_rows: 850
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'pos_tags'],
        num_rows: 864
    })
})

In [4]:
# Code obtained from: https://huggingface.co/learn/nlp-course/chapter7/2?fw=pt

# Pre-processing the data and tokenize

def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))
    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

tokenized_datasets = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset["train"].column_names,
)


Map:   0%|          | 0/1992 [00:00<?, ? examples/s]

Map:   0%|          | 0/850 [00:00<?, ? examples/s]

Map:   0%|          | 0/864 [00:00<?, ? examples/s]

Now, the pre-processing of the data is finished

# Question 3
Fine-tune a model with the default hyperparameter settings on the train set and evaluate the model on the test set. These are your baseline results.

In [5]:
# Code obtained from: https://huggingface.co/learn/nlp-course/chapter7/2?fw=pt
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer,
    return_tensors="tf"
)

In [6]:
# Code obtained from: https://huggingface.co/learn/nlp-course/chapter7/2?fw=pt

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=16,
)

tf_eval_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

tf_test_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

In [7]:
# Code obtained from: https://huggingface.co/learn/nlp-course/chapter7/2?fw=pt
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [8]:
# Code obtained from: https://huggingface.co/learn/nlp-course/chapter7/2?fw=pt
from transformers import TFAutoModelForTokenClassification

model = TFAutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

All PyTorch model weights were used when initializing TFBertForTokenClassification.

Some weights or buffers of the TF 2.0 model TFBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# Code obtained from: https://huggingface.co/learn/nlp-course/chapter7/2?fw=pt
from transformers import create_optimizer
import tensorflow as tf

# Train in mixed-precision float16
# Comment this line out if you're using a GPU that will not benefit from this
tf.keras.mixed_precision.set_global_policy("mixed_float16")

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_epochs = 3
num_train_steps = len(tf_train_dataset) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)


The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [10]:
# Code obtained from: https://huggingface.co/learn/nlp-course/chapter7/2?fw=pt
from transformers.keras_callbacks import PushToHubCallback

model.fit(
    tf_train_dataset,
    validation_data=tf_eval_dataset,
    epochs=num_epochs,
)

Epoch 1/3
125/125 [==============================] - 1182s 9s/step - loss: 0.4969 - val_loss: 0.2784
Epoch 2/3
125/125 [==============================] - 1051s 8s/step - loss: 0.1566 - val_loss: 0.2392
Epoch 3/3
125/125 [==============================] - 1022s 8s/step - loss: 0.1124 - val_loss: 0.2475


In [11]:
# Code obtained from: https://huggingface.co/learn/nlp-course/chapter7/2?fw=pt
import evaluate

metric = evaluate.load("seqeval")

labels = dataset["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
predictions = labels.copy()
metric.compute(predictions=[predictions], references=[labels])

c:\Users\Juliu\AppData\Local\Programs\Python\Python311\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Juliu\AppData\Local\Programs\Python\Python311\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Juliu\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
c:\Users\Juliu\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret

{'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 1.0}

In [ ]:
# Code obtained from: https://huggingface.co/learn/nlp-course/chapter7/2?fw=pt
import evaluate

metric = evaluate.load("seqeval")

label_names = ['O', 'B-PER', 'I-PER', 'B-LOC', 'I-LOC', 'B-ART', 'I-ART', 'I-MAT', 'B-MAT', 'I-CON', 'B-CON', 'I-SPE', 'B-SPE']

# Code obtained from: https://huggingface.co/learn/nlp-course/chapter7/2?fw=pt
import numpy as np

all_predictions = []
all_labels = []
for batch in tf_test_dataset:
    logits = model.predict_on_batch(batch)["logits"]
    labels = batch["labels"]
    predictions = np.argmax(logits, axis=-1)
    for prediction, label in zip(predictions, labels):
        for predicted_idx, label_idx in zip(prediction, label):
            if label_idx == -100:
                continue
            all_predictions.append(label_names[predicted_idx])
            all_labels.append(label_names[label_idx])

metric.compute(predictions=[all_predictions], references=[all_labels])

c:\Users\Juliu\AppData\Local\Programs\Python\Python311\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'ART': {'precision': 0.35074626865671643,
  'recall': 0.5595238095238095,
  'f1': 0.4311926605504587,
  'number': 168},
 'CON': {'precision': 0.3875,
  'recall': 0.5740740740740741,
  'f1': 0.4626865671641791,
  'number': 216},
 'LOC': {'precision': 0.5439560439560439,
  'recall': 0.6875,
  'f1': 0.6073619631901841,
  'number': 144},
 'MAT': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 107},
 'PER': {'precision': 0.7529761904761905,
  'recall': 0.8939929328621908,
  'f1': 0.8174474959612278,
  'number': 283},
 'SPE': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2},
 'overall_precision': 0.5144404332129964,
 'overall_recall': 0.6195652173913043,
 'overall_f1': 0.5621301775147929,
 'overall_accuracy': 0.9561058220432076}

: 

# Question 4
Set up hyperparameter optimization (HPO), use the val set as validation. Optimize at least three hyperparameters (learning rate, batch size and weight decay). You can choose your own way to implement this and select your own grid. After the model has been optimized, evaluate the result on the test set.

In [16]:
import pandas as pd

#learning_rates = [1e-5, 2e-5, 3e-5]
#batch_sizes = [8, 16, 32]
#weight_decays = [0.01, 0.001, 0.0001]

# Define hyperparameter grids
learning_rates = [1e-5, 2e-5]
batch_sizes = [16, 32]
weight_decays = [0.01, 0.001]

# Initialize an empty DataFrame to store results
results = pd.DataFrame(columns=["learning_rate", "batch_size", "weight_decay", "val_loss"])
n = 1
# Hyperparameter optimization loop
for lr in learning_rates:
    for batch_size in batch_sizes:
        for weight_decay in weight_decays:
            # Create tf datasets with the current batch size
            tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
                columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
                collate_fn=data_collator,
                shuffle=True,
                batch_size=batch_size,
            )
            tf_eval_dataset = tokenized_datasets["validation"].to_tf_dataset(
                columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
                collate_fn=data_collator,
                shuffle=False,
                batch_size=batch_size,
            )

            # Set up optimizer with the current learning rate and weight decay
            optimizer, schedule = create_optimizer(
                init_lr=lr,
                num_warmup_steps=0,
                num_train_steps=len(tf_train_dataset) * num_epochs,
                weight_decay_rate=weight_decay,
            )
            model.compile(optimizer=optimizer)

            # Train the model
            history = model.fit(
                tf_train_dataset,
                validation_data=tf_eval_dataset,
                epochs=num_epochs,
            )

            # Get the validation loss for the final epoch
            val_loss = history.history['val_loss'][-1]

            # Append the results as a new DataFrame and concatenate
            new_row = pd.DataFrame({
                "learning_rate": [lr],
                "batch_size": [batch_size],
                "weight_decay": [weight_decay],
                "val_loss": [val_loss]
            })
            results = pd.concat([results, new_row], ignore_index=True)
            print('Test {n} out of 6 has concluded')
            n += 1

# Display all results
print(results)

# After optimization, evaluate on test set with best hyperparameters
best_params = results.loc[results['val_loss'].idxmin()]
print("Best hyperparameters:", best_params)

Epoch 1/3
125/125 [==============================] - 1098s 9s/step - loss: 0.5080 - val_loss: 0.3464
Epoch 2/3
125/125 [==============================] - 1035s 8s/step - loss: 0.2218 - val_loss: 0.2752
Epoch 3/3
125/125 [==============================] - 995s 8s/step - loss: 0.1667 - val_loss: 0.2655
Test {n} out of 6 has concluded


C:\Users\Juliu\AppData\Local\Temp\ipykernel_17820\2365670061.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, new_row], ignore_index=True)


Epoch 1/3
125/125 [==============================] - 1027s 8s/step - loss: 0.1379 - val_loss: 0.2417
Epoch 2/3
125/125 [==============================] - 995s 8s/step - loss: 0.0987 - val_loss: 0.2437
Epoch 3/3
125/125 [==============================] - 1000s 8s/step - loss: 0.0840 - val_loss: 0.2493
Test {n} out of 6 has concluded
Epoch 1/3
63/63 [==============================] - 1191s 19s/step - loss: 0.0811 - val_loss: 0.2399
Epoch 2/3
63/63 [==============================] - 1148s 18s/step - loss: 0.0653 - val_loss: 0.2400
Epoch 3/3
63/63 [==============================] - 1160s 19s/step - loss: 0.0581 - val_loss: 0.2470
Test {n} out of 6 has concluded
Epoch 1/3
63/63 [==============================] - 1209s 19s/step - loss: 0.0577 - val_loss: 0.2569
Epoch 2/3
63/63 [==============================] - 1195s 19s/step - loss: 0.0471 - val_loss: 0.2679
Epoch 3/3
63/63 [==============================] - 1222s 19s/step - loss: 0.0408 - val_loss: 0.2720
Test {n} out of 6 has concluded
Ep

In [28]:
# Re-train with best hyperparameters and evaluate on test set
best_lr = float(best_params["learning_rate"])
best_batch_size = int(best_params["batch_size"])
best_weight_decay = float(best_params["weight_decay"])

print(best_lr, best_batch_size, best_weight_decay)

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=best_batch_size,
)
tf_eval_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=best_batch_size,
)

            # Set up optimizer with the current learning rate and weight decay
optimizer, schedule = create_optimizer(
    init_lr=best_lr,
    num_warmup_steps=0,
    num_train_steps=len(tf_train_dataset) * num_epochs,
    weight_decay_rate=best_weight_decay,
)

model.compile(optimizer=optimizer)

history = model.fit(
    tf_train_dataset,
    validation_data=tf_eval_dataset,
    epochs=num_epochs,
)

# Evaluate on test set
all_predictions = []
all_labels = []
for batch in tf_test_dataset:
    logits = model.predict_on_batch(batch)["logits"]
    labels = batch["labels"]
    predictions = np.argmax(logits, axis=-1)
    for prediction, label in zip(predictions, labels):
        for predicted_idx, label_idx in zip(prediction, label):
            if label_idx == -100:
                continue
            all_predictions.append(label_names[predicted_idx])
            all_labels.append(label_names[label_idx])

test_metric = metric.compute(predictions=[all_predictions], references=[all_labels])
print("Test set performance:", test_metric)

1e-05 32 0.01
Epoch 1/3
63/63 [==============================] - 1278s 20s/step - loss: 0.0060 - val_loss: 0.3760
Epoch 2/3
63/63 [==============================] - 1225s 20s/step - loss: 0.0048 - val_loss: 0.3965
Epoch 3/3
63/63 [==============================] - 1219s 19s/step - loss: 0.0036 - val_loss: 0.3936


NameError: name 'np' is not defined

# Question 5
Extend the evaluation function so that it shows the Precision, Recall and F-score for each of the entity types (location, artefact, etc.) on the test set. Include the metrics for the B-label of the entity type, the I-label, and the full entities.

# Question 6
Look up the definitions of macro- and micro-average scores and compute the macro- and micro average F1 scores over all entities.

In [31]:
# Code obtained from: https://huggingface.co/learn/nlp-course/chapter7/2?fw=pt
import numpy as np
import evaluate
metric = evaluate.load("seqeval")

all_predictions = []
all_labels = []
for batch in tf_eval_dataset:
    logits = model.predict_on_batch(batch)["logits"]
    labels = batch["labels"]
    predictions = np.argmax(logits, axis=-1)
    for prediction, label in zip(predictions, labels):
        for predicted_idx, label_idx in zip(prediction, label):
            if label_idx == -100:
                continue
            all_predictions.append(label_names[predicted_idx])
            all_labels.append(label_names[label_idx])
metric.compute(predictions=[all_predictions], references=[all_labels])

{'ART': {'precision': 0.41643835616438357,
  'recall': 0.6031746031746031,
  'f1': 0.4927066450567262,
  'number': 252},
 'CON': {'precision': 0.37346437346437344,
  'recall': 0.6495726495726496,
  'f1': 0.4742589703588144,
  'number': 234},
 'LOC': {'precision': 0.6491228070175439,
  'recall': 0.7449664429530202,
  'f1': 0.6937500000000001,
  'number': 149},
 'MAT': {'precision': 0.43137254901960786,
  'recall': 0.3142857142857143,
  'f1': 0.3636363636363637,
  'number': 70},
 'PER': {'precision': 0.6634844868735084,
  'recall': 0.8249258160237388,
  'f1': 0.7354497354497354,
  'number': 337},
 'SPE': {'precision': 0.4945054945054945,
  'recall': 0.7627118644067796,
  'f1': 0.6,
  'number': 59},
 'overall_precision': 0.5053191489361702,
 'overall_recall': 0.6902815622161671,
 'overall_f1': 0.5834932821497122,
 'overall_accuracy': 0.9422615673106275}

In [32]:
import numpy as np
import evaluate
from seqeval.metrics import classification_report as seqeval_classification_report

# Load seqeval metric
metric = evaluate.load("seqeval")

all_predictions = []
all_labels = []

# Evaluate on the test set
for batch in tf_test_dataset:
    logits = model.predict_on_batch(batch)["logits"]
    batch_labels = batch["labels"].numpy()  # Avoid overwriting variable 'labels'
    batch_predictions = np.argmax(logits, axis=-1)  # Avoid overwriting variable 'predictions'

    for pred, true_label in zip(batch_predictions, batch_labels):
        pred_sequence = []
        label_sequence = []
        for predicted_idx, label_idx in zip(pred, true_label):
            if label_idx == -100:  # Skip padding
                continue
            pred_label = label_names[predicted_idx]
            true_label_str = label_names[label_idx]
            pred_sequence.append(pred_label)
            label_sequence.append(true_label_str)
        all_predictions.append(pred_sequence)
        all_labels.append(label_sequence)

# Compute overall metrics
results = metric.compute(predictions=all_predictions, references=all_labels)

print("Overall Metrics:")
print(f"Precision: {results['overall_precision']:.4f}")
print(f"Recall: {results['overall_recall']:.4f}")
print(f"F1-Score: {results['overall_f1']:.4f}")
print(f"Accuracy: {results['overall_accuracy']:.4f}")

# Use seqeval's classification_report to get detailed per-entity metrics
print("\nPer-Entity Metrics:")
print(seqeval_classification_report(all_labels, all_predictions))

Overall Metrics:
Precision: 0.5683
Recall: 0.7326
F1-Score: 0.6401
Accuracy: 0.9569

Per-Entity Metrics:
              precision    recall  f1-score   support

         ART       0.48      0.66      0.56       168
         CON       0.45      0.67      0.53       216
         LOC       0.60      0.78      0.68       144
         MAT       0.63      0.49      0.55       107
         PER       0.71      0.89      0.79       283
         SPE       0.10      0.50      0.17         2

   micro avg       0.57      0.73      0.64       920
   macro avg       0.50      0.67      0.55       920
weighted avg       0.58      0.73      0.64       920



In [ ]:
from collections import Counter

# Count label occurrences in the training dataset
label_counter = Counter()
for batch in tf_train_dataset:
    labels = batch["labels"].numpy()
    for label_seq in labels:
        for label in label_seq:
            if label != -100:  # Exclude padding labels
                label_counter[label_names[label]] += 1

print("Label distribution in the training set:")
for label, count in label_counter.items():
    print(f"{label}: {count}")

Label distribution in the training set:
O: 57424
B-ART: 1000
I-ART: 965
B-PER: 1213
I-PER: 1631
B-CON: 1423
B-MAT: 185
I-MAT: 26
I-CON: 153
B-LOC: 342
I-LOC: 679
B-SPE: 184
I-SPE: 9
